Для решения задач выбрала библиотеку `pandas`, так как с помощью неё можно легко перенести исходный файл в датафрейм и с помощью математических и аггрегирующих функций решить предложенные задачи. На мой взгляд этот способ быстрее и удобнее формул в экселе.

Загрузила файл `Тестовый датасет.xlsx` в датафрейм, вывела `df.shape`, чтобы проверить количество строк и колонок

In [19]:
import pandas as pd
df = pd.read_excel('Тестовый датасет.xlsx')
print(df.shape)

(2000, 9)


Посчитала число уникальных пользователей в день. Для этого создала датафрейм `daily_users`. Сгруппировала данные по `session_start`, извлекла дату с помощью `pd.to_datetime().dt.date`, использовала `nunique()` для уникальности пользователей, вывела результат в консоль без индекса через `to_string(index=False)`.

In [37]:
daily_users = (
    df.groupby(pd.to_datetime(df['session_start']).dt.date)['user_id']
    .nunique()
    .reset_index()
    .rename(columns = {'user_id': 'users'})
)
print(daily_users.to_string(index=False))

session_start  users
   2025-01-01    417
   2025-01-02     90
   2025-01-03      1


Посчитала количество сессий по платформам. Для этого использовала `value_counts(dropna=True)` для подсчёта уникальных значений в колонке `platform` исключая `NaN`.

In [17]:
platform_counts = df['platform'].value_counts(dropna=True).reset_index()
platform_counts.columns = ['platform', 'sessions']
print(platform_counts.to_string(index=False))

platform  sessions
     IOS       688
      PC       657
 Android       655


Посчитала среднюю продолжительность сессий по платформам в минутах.
Для этого вычислила длительность каждой сессии как разность `session_end - session_start` в секундах через `pd.to_datetime()`, перевела в минуты для удобства делением на 60. Отфильтровала сессии с положительной длительностью `(query('duration > 0'))`, сгруппировала по платформе, посчитала среднее через `mean()`.

In [13]:
durations = (
    pd.to_datetime(df['session_end']) -
    pd.to_datetime(df['session_start'])
).dt.total_seconds() / 60

avg_duration = (
    df[['platform']].assign(duration=durations)
    .query('duration > 0')
    .groupby('platform')['duration']
    .mean()
    .reset_index()
    .rename(columns={'duration':'avg_duration_min'})
)
print(avg_duration.to_string(index=False))


platform  avg_duration_min
 Android         42.867176
     IOS         47.344477
      PC         43.773212


Посчитала долю новых пользователей среди записей с непустым полем. Для этого с `dropna(subset=['is_new_user'])` исключила строки с NaN в колонке новых пользователей. Вычислила долю `TRUE` значений через `mean()`, так как `pandas` интерпретирует `TRUE` как `1.0`.

In [40]:
df_valid_users = df.dropna(subset=['is_new_user'])
new_user_share = df_valid_users['is_new_user'].mean()
print(f"Доля новых пользователей:\n{new_user_share}")

Доля новых пользователей:
0.29712272460364064


Посчитала медианную продолжительность сессий для новых и старых пользователей в минутах. Для этого удалила строки с `NaN`, отфильтровала новых пользователей через `astype(bool)` и старых через отрицание `~`, посчитала медианы через `median()` для каждой группы.

In [15]:
df_temp = df.assign(duration=durations).dropna(subset=['duration', 'is_new_user'])

median_new = df_temp[df_temp['is_new_user'].astype(bool)]['duration'].median()
median_old = df_temp[~df_temp['is_new_user'].astype(bool)]['duration'].median()

median_table = pd.DataFrame({
    'user_type': ['new', 'old'],
    'median_duration_min': [median_new, median_old]
})
print(median_table.to_string(index=False))

user_type  median_duration_min
      new                 31.5
      old                 32.0


Посчитала суммарное время сессий пользователей из России в минутах. Отфильтровала по `country == 'RU'`. Вычислила длительность каждой сессии, суммировала все длительности через `sum()`, перевела секунды в минуты делением на 60.

In [14]:
df_ru = df[df['country'] == 'RU']

durations_ru = (
    pd.to_datetime(df_ru['session_end']) -
    pd.to_datetime(df_ru['session_start'])
).dt.total_seconds()

ru_total_time = durations_ru.sum() / 60
print(f"Суммарное отыгранное время:\n{ru_total_time} мин")

Суммарное отыгранное время:
21589.0 мин


## Интерпретация результатов

`Критическая проблема с удержанием пользователей`

C 417 пользователей в 1 день активность снизилась до 90 во 2 день, (3 день исключим так как последние измерения были в 1 ночи)
Вероятно был массовый приток аудитории, после которого произошла потеря 78% пользователей

`Равномерное распределение по платформам`

По 33% у PC и Android, 34% у IOS
Контент удобен на всех платформах

`Равномерная продолжительность сессий по платформах`

IOS пользователи проводят чуть больше времени, возможно интерфейс удобнее

`Новые и старые пользователи играют одинаково`

Только по общему времени российской аудитории тяжело сделать вывод не сравнивая с общим числом или с другими странами

`Преимущества:`

Кроссплатформенность работает, длинные сессии пользователей от 43 минут, приток новых пользователей 30%

`Недостатки:`

Пользователи не возвращаются

## Предложения

Retention - сколько вернулось на следующий день, через неделю, месяц

Воронка - где теряются пользователи по этапам (регистрация, первая игра, вторая игра)

Количество сессий на игрока в день

Глубина - сколько матчей/уровней за сессию

Активные игроки - выделить общие черты

Отток - кто не вернулся через n дней, выделить общие черты

Краши - сколько сессий падает на iOS/Android/PC количественно и в процентах

Среднее время сессии

Можно смотреть отдельно по платформам, по странам, по году регистрации

## SQL-запрос
Написала SQL-запрос для создания представления `user_activity`. Таблица `sessions` содержит все необходимые колонки кроме `is_new_user` и `country`. Соединением `LEFT JOIN` с таблицей `users` по `user_id` получаем поле `country`. `is_new_user` вычисляется через подзапрос: для каждой сессии проверяем, равна ли её дата минимальной дате сессий этого же пользователя. В конце отсортировала по `session_id` как в исходном датасете.

In [ ]:
%%sql
SELECT
    s.session_id,
    s.user_id,
    CASE
        WHEN DATE(s.session_start) = (
            SELECT MIN(DATE(s2.session_start))
            FROM sessions s2
            WHERE s2.user_id = s.user_id
        ) THEN 'TRUE'
        ELSE 'FALSE'
    END AS is_new_user,
    u.country,
    s.session_start,
    s.session_end,
    s.platform,
    s.game_id,
    s.session_status
FROM sessions s
LEFT JOIN users u ON s.user_id = u.user_id
ORDER BY s.session_id;
